# 10 — Custom User Profile

Four ways to build a user profile and get personalized PSX news recommendations.

| Mode | Input | Profile method |
|------|-------|----------------|
| 1 | Choose sectors | Sample articles → average embeddings |
| 2 | Free keywords | Encode keywords directly with SBERT |
| 3 | Free keywords | Find matching articles → average embeddings |
| 4 | Article history | TF-IDF summary → encode with SBERT |

**Section A** — Interactive widgets (pick your mode)

**Section B** — Side-by-side comparison of all 4 modes on the same input

In [5]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

df_pool   = pd.read_csv('../data/processed/article_pool.csv')
emb_sbert = np.load('../data/processed/embeddings_pool_sbert_mpnet.npy')

print('Loading SBERT...')
sbert = SentenceTransformer('all-mpnet-base-v2')
print(f'Ready — pool: {len(df_pool)} articles, embeddings: {emb_sbert.shape}')

Loading SBERT...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Ready — pool: 14213 articles, embeddings: (14213, 768)


In [6]:
# ── Shared helpers ────────────────────────────────────────────────────────────

TAG_COLORS = {
    'Construction': '#f5a623',
    'Banking':      '#4a90d9',
    'Energy':       '#e74c3c',
    'Other':        '#95a5a6'
}

def get_recommendations(profile, exclude_indices, k):
    scores  = cosine_similarity([profile], emb_sbert)[0]
    results = df_pool.copy()
    results['article_id'] = results.index
    results['score']      = scores
    results = results[~results['article_id'].isin(exclude_indices)]
    return results.sort_values('score', ascending=False).head(k).reset_index(drop=True)

def article_card(rank, headline, tag, score=None):
    color      = TAG_COLORS.get(tag, '#95a5a6')
    prefix     = f'#{rank}' if rank else '📖'
    score_line = f'<p style="color:gray;font-size:11px;margin:2px 0 0 0;">Score: {score:.4f}</p>' if score is not None else ''
    return f'''
    <div style="border:1px solid #ddd;border-radius:6px;padding:10px;margin:5px 0;">
        <span style="background:{color};color:white;padding:2px 7px;
              border-radius:4px;font-size:11px;">{prefix} {tag}</span>
        <p style="margin:6px 0 0 0;font-size:13px;">{str(headline)[:120]}</p>
        {score_line}
    </div>'''

# ── Profile building — 4 methods ─────────────────────────────────────────────

def mode1_sectors_to_articles(sectors, n_seed, random_seed):
    """Sample articles from chosen sectors, average their embeddings."""
    pool = df_pool[df_pool['primary_tag'].isin(sectors)]
    if pool.empty:
        return None, pd.DataFrame()
    sample = pool.sample(n=min(n_seed, len(pool)), random_state=random_seed).reset_index()
    profile = emb_sbert[sample['index'].tolist()].mean(axis=0)
    return profile, sample

def mode2_keywords_direct(keywords_text):
    """Encode keywords directly with SBERT — no article lookup."""
    kws = [k.strip() for k in keywords_text.replace(',', ' ').split() if k.strip()]
    if not kws:
        return None
    vecs = sbert.encode(kws, show_progress_bar=False)
    return vecs.mean(axis=0)

def mode3_keywords_to_articles(keywords_text, n_seed, random_seed):
    """Find articles matching keywords, average their embeddings."""
    kws = [k.strip().lower() for k in keywords_text.replace(',', ' ').split() if k.strip()]
    if not kws:
        return None, pd.DataFrame()
    mask = df_pool['headline'].str.lower().apply(
        lambda h: any(kw in str(h) for kw in kws)
    )
    pool = df_pool[mask]
    if pool.empty:
        return None, pd.DataFrame()
    sample = pool.sample(n=min(n_seed, len(pool)), random_state=random_seed).reset_index()
    profile = emb_sbert[sample['index'].tolist()].mean(axis=0)
    return profile, sample

def mode4_history_to_summary(article_indices):
    """TF-IDF summary of article history → encode with SBERT."""
    headlines = df_pool.iloc[article_indices]['headline'].dropna().tolist()
    if not headlines:
        return None, ''
    tfidf = TfidfVectorizer(max_features=10, stop_words='english')
    tfidf.fit(headlines)
    top_words = list(tfidf.vocabulary_.keys())
    summary   = ' '.join(top_words)
    profile   = sbert.encode([summary], show_progress_bar=False)[0]
    return profile, summary

print('Helpers ready')

Helpers ready


## Section A — Interactive Demo

Choose a mode, configure your inputs, and hit Recommend.

In [7]:
# ── Mode selector ─────────────────────────────────────────────────────────────
mode_toggle = widgets.ToggleButtons(
    options=[
        ('Mode 1 — Sectors → Articles', 1),
        ('Mode 2 — Keywords direct', 2),
        ('Mode 3 — Keywords → Articles', 3),
        ('Mode 4 — History → Summary', 4),
    ],
    button_style='info'
)

# ── Shared controls ───────────────────────────────────────────────────────────
k_slider = widgets.IntSlider(value=3, min=1, max=10, description='Top-K:', style={'description_width':'initial'})
seed_slider = widgets.IntSlider(value=42, min=1, max=100, description='Random seed:', style={'description_width':'initial'})
n_seed_slider = widgets.IntSlider(value=5, min=2, max=15, description='Seed articles:', style={'description_width':'initial'})

# ── Mode-specific controls ────────────────────────────────────────────────────
sector_box = widgets.SelectMultiple(
    options=['Construction', 'Banking', 'Energy'], value=['Banking'],
    description='Sectors:', rows=3, layout=widgets.Layout(width='220px')
)
keywords_box = widgets.Textarea(
    placeholder='e.g. cement, OGDC, interest rate, IMF, Lucky Cement...',
    description='Keywords:', layout=widgets.Layout(width='380px', height='70px')
)
history_box = widgets.Textarea(
    placeholder='Paste article indices (comma-separated), e.g. 0, 5, 42, 100',
    description='Article IDs:', layout=widgets.Layout(width='380px', height='70px')
)

# Panels — shown/hidden based on mode
panel1 = widgets.VBox([sector_box, n_seed_slider, seed_slider])
panel2 = widgets.VBox([keywords_box])
panel3 = widgets.VBox([keywords_box, n_seed_slider, seed_slider])
panel4 = widgets.VBox([history_box])

panel_container = widgets.VBox([panel1])
button = widgets.Button(description='🔍 Recommend', button_style='primary',
                        layout=widgets.Layout(width='200px', height='38px'))
output = widgets.Output()

def update_panel(change):
    mode = mode_toggle.value
    panel_container.children = [{
        1: panel1, 2: panel2, 3: panel3, 4: panel4
    }[mode]]

mode_toggle.observe(update_panel, names='value')

# ── Recommend handler ─────────────────────────────────────────────────────────
def on_recommend(b):
    with output:
        clear_output(wait=True)
        mode = mode_toggle.value
        k    = k_slider.value
        seed_articles = pd.DataFrame()
        profile       = None
        summary_text  = ''

        if mode == 1:
            sectors = list(sector_box.value)
            if not sectors:
                display(HTML('<p style="color:red">⚠️ Select at least one sector.</p>'))
                return
            profile, seed_articles = mode1_sectors_to_articles(sectors, n_seed_slider.value, seed_slider.value)

        elif mode == 2:
            if not keywords_box.value.strip():
                display(HTML('<p style="color:red">⚠️ Enter at least one keyword.</p>'))
                return
            profile = mode2_keywords_direct(keywords_box.value)

        elif mode == 3:
            if not keywords_box.value.strip():
                display(HTML('<p style="color:red">⚠️ Enter at least one keyword.</p>'))
                return
            profile, seed_articles = mode3_keywords_to_articles(keywords_box.value, n_seed_slider.value, seed_slider.value)
            if seed_articles.empty:
                display(HTML('<p style="color:red">⚠️ No articles found matching your keywords.</p>'))
                return

        elif mode == 4:
            try:
                ids = [int(x.strip()) for x in history_box.value.split(',') if x.strip()]
            except:
                display(HTML('<p style="color:red">⚠️ Invalid article IDs — use comma-separated integers.</p>'))
                return
            if not ids:
                display(HTML('<p style="color:red">⚠️ Enter at least one article ID.</p>'))
                return
            ids = [i for i in ids if i < len(df_pool)]
            profile, summary_text = mode4_history_to_summary(ids)
            seed_articles = df_pool.iloc[ids].reset_index()

        if profile is None:
            display(HTML('<p style="color:red">⚠️ Could not build profile.</p>'))
            return

        exclude = seed_articles['index'].tolist() if not seed_articles.empty else []
        recs    = get_recommendations(profile, exclude, k)

        # ── Output HTML ───────────────────────────────────────────────────────
        mode_labels = {
            1: 'Sectors → Articles → Avg embeddings',
            2: 'Keywords → SBERT direct',
            3: 'Keywords → Articles → Avg embeddings',
            4: 'History → TF-IDF summary → SBERT'
        }
        html = f'<p style="color:gray;font-size:12px;">Mode {mode}: {mode_labels[mode]}</p>'

        if not seed_articles.empty:
            html += f'<h3>📚 Reading History / Seed Articles ({len(seed_articles)})</h3>'
            for _, row in seed_articles.iterrows():
                html += article_card(None, row['headline'], row['primary_tag'])

        if summary_text:
            html += f'<p><b>TF-IDF profile summary:</b> <i>"{summary_text}"</i></p>'

        html += f'<h3>✨ Top-{k} Recommendations</h3>'
        for i, row in recs.iterrows():
            html += article_card(i+1, row['headline'], row['primary_tag'], row['score'])

        display(HTML(html))

button.on_click(on_recommend)

display(HTML('<h3>⚙️ Configure</h3>'))
display(mode_toggle)
display(widgets.HBox([panel_container, widgets.VBox([k_slider, button])]))
display(output)

ToggleButtons(button_style='info', options=(('Mode 1 — Sectors → Articles', 1), ('Mode 2 — Keywords direct', 2…

Output()

## Section B — Side-by-side Comparison

Same input, 4 profile methods. Shows how the choice of profile construction affects recommendations.

In [8]:
# ── Fixed input for comparison ────────────────────────────────────────────────
# Change these to compare the 4 modes on your own inputs

SECTORS      = ['Banking']         # used by modes 1 and as reference
KEYWORDS     = 'NBP HBL interest rate deposits'  # used by modes 2 and 3
HISTORY_IDS  = [20, 21, 22, 23, 24]              # used by mode 4 (article indices)
N_SEED       = 5
RANDOM_SEED  = 42
K            = 3

print(f'Comparing 4 modes with:')
print(f'  Sectors:  {SECTORS}')
print(f'  Keywords: {KEYWORDS}')
print(f'  History:  articles {HISTORY_IDS}')

Comparing 4 modes with:
  Sectors:  ['Banking']
  Keywords: NBP HBL interest rate deposits
  History:  articles [20, 21, 22, 23, 24]


In [9]:
# ── Run all 4 modes ───────────────────────────────────────────────────────────

results_all = {}
seeds_all   = {}
summaries   = {}

# Mode 1
p1, s1 = mode1_sectors_to_articles(SECTORS, N_SEED, RANDOM_SEED)
results_all[1] = get_recommendations(p1, s1['index'].tolist(), K)
seeds_all[1]   = s1

# Mode 2
p2 = mode2_keywords_direct(KEYWORDS)
results_all[2] = get_recommendations(p2, [], K)

# Mode 3
p3, s3 = mode3_keywords_to_articles(KEYWORDS, N_SEED, RANDOM_SEED)
results_all[3] = get_recommendations(p3, s3['index'].tolist() if not s3.empty else [], K)
seeds_all[3]   = s3

# Mode 4
valid_ids = [i for i in HISTORY_IDS if i < len(df_pool)]
p4, summary = mode4_history_to_summary(valid_ids)
results_all[4] = get_recommendations(p4, valid_ids, K)
summaries[4]   = summary

print('All 4 modes computed')

All 4 modes computed


In [10]:
# ── Display side by side ──────────────────────────────────────────────────────

mode_labels = {
    1: f'Mode 1<br><small>Sectors → Articles</small>',
    2: f'Mode 2<br><small>Keywords direct</small>',
    3: f'Mode 3<br><small>Keywords → Articles</small>',
    4: f'Mode 4<br><small>History → TF-IDF</small>',
}

html = '<table style="width:100%;border-collapse:collapse;">'
html += '<tr>'
for m in [1, 2, 3, 4]:
    html += f'<th style="padding:10px;background:#f0f0f0;border:1px solid #ddd;text-align:center;">{mode_labels[m]}</th>'
html += '</tr><tr>'

for m in [1, 2, 3, 4]:
    cell = '<td style="padding:8px;border:1px solid #ddd;vertical-align:top;">'

    # Seed info
    if m in seeds_all and not seeds_all[m].empty:
        cell += f'<p style="font-size:11px;color:gray;">Profile from {len(seeds_all[m])} articles:</p>'
        for _, row in seeds_all[m].iterrows():
            color = TAG_COLORS.get(row['primary_tag'], '#95a5a6')
            cell += f'<p style="font-size:11px;margin:2px 0;"><span style="background:{color};color:white;padding:1px 5px;border-radius:3px;">{row["primary_tag"]}</span> {str(row["headline"])[:60]}...</p>'
    elif m == 2:
        cell += f'<p style="font-size:11px;color:gray;">Keywords: <i>{KEYWORDS}</i></p>'
    elif m == 4:
        cell += f'<p style="font-size:11px;color:gray;">TF-IDF summary: <i>"{summaries[4]}"</i></p>'

    cell += '<hr style="margin:6px 0;">'
    cell += f'<b style="font-size:12px;">Top-{K} Recommendations</b>'

    for i, row in results_all[m].iterrows():
        color = TAG_COLORS.get(row['primary_tag'], '#95a5a6')
        cell += f'''
        <div style="border:1px solid #eee;border-radius:5px;padding:7px;margin:5px 0;">
            <span style="background:{color};color:white;padding:1px 6px;border-radius:3px;font-size:11px;">#{i+1} {row["primary_tag"]}</span>
            <p style="font-size:12px;margin:4px 0 2px 0;">{str(row["headline"])[:80]}...</p>
            <p style="font-size:11px;color:gray;margin:0;">Score: {row["score"]:.4f}</p>
        </div>'''

    cell += '</td>'
    html += cell

html += '</tr></table>'
display(HTML(html))

Mode 1Sectors → Articles,Mode 2Keywords direct,Mode 3Keywords → Articles,Mode 4History → TF-IDF
"Profile from 5 articles:Banking ['Axact, a bank led to closure of HBL branch in New York, MP...Banking ['Govt raises $542m through UBL, PPL sell-off', 'MCB Bank, T...Banking ['UBL Fund declares payout']...Banking ['NAB fails to arrest former NBP president\xa0as SHC dismiss...Banking ['NBP receives Consumers’ Choice Award']...Top-3 Recommendations #1 Banking ['Loan write-off issue politicised: NBP chief', 'UBL Fund Managers announces fin... Score: 0.8011 #2 Banking ['UBL funds announce payouts']... Score: 0.7667 #3 Banking ['UBL announces 10pc cash dividend', 'NBP aims to be top bank of region']... Score: 0.7658","Keywords: NBP HBL interest rate depositsTop-3 Recommendations #1 Banking ['HBL posts profits']... Score: 0.5042 #2 Banking ['NBP’s profit up 42pc']... Score: 0.4859 #3 Banking ['NBP profits up 173pc', 'HBL, Byco sign ATM partnership deal']... Score: 0.4846","Profile from 5 articles:Banking ['NBP ratings reaffirmed']...Banking ['HBL IPO starts today']...Other ['Cotton export yet to kick off due to procedural discrepanc...Banking ['HBL enters in alliance with SSC']...Other ['Textile mill-owners in a fix on rupee revaluation', 'Moder...Top-3 Recommendations #1 Banking ['HBL, SEFAM sign agreement', 'KSE-100 Index down as MCB sustains selling pressu... Score: 0.7413 #2 Other ['KESC, Textile City sign power supply agreement', 'Cotton trading remains firm'... Score: 0.7363 #3 Banking ['HBL IPO price irks investors', 'Consortium selected for NBP GDR']... Score: 0.7346","TF-IDF summary: ""traders gains range kse likely maintain 000 level points stocks""Top-3 Recommendations #1 Other ['Range-bound KSE gains 55 points amid dull trading']... Score: 0.6805 #2 Other ['KSE crosses 36,000 points level as buyers eye robust earnings']... Score: 0.6733 #3 Other ['Regulated KSE gains nine points in range-bound trading']... Score: 0.6718"
